# Imports

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline
import pandas as pd 
import time
from math import log

------------------------------

The main idea is not to deal with index but rather the heading of the columns. Therefore we use pandas.

# Helper Function

In [2]:
def entropy(target_col, n_class):
    '''
    Compute Entropy
    @param target_col: the columns of the target value
    @param n_class: the number of class as log base
    
    return: entropy 
    '''
    elements,counts = np.unique(target_col,return_counts = True)
    ent = 0
    for i in range(len(elements)):
        ent += (-counts[i]/np.sum(counts))*log(counts[i]/np.sum(counts), n_class)
    return ent    

def gain(data, att_name, n_class, target_col_name = 'class'):
    
    assert type(att_name)==str, "Expected type str for att_name!"
    
    total_entropy = entropy(data[target_col_name], n_class)
    #return non-duplicate values from the column and it's frequency
    vals,counts= np.unique(data[att_name],return_counts=True)
    w_entropy = 0
    for i in range(len(vals)):
        #filter out the class with its corresponding attributes w.r.t features
        att = data.where(data[att_name]==vals[i]).dropna()[target_col_name]
        w_entropy += (counts[i]/np.sum(counts))*entropy(att, n_class)
    return total_entropy- w_entropy

def gain02(data, att_name, n_class, total_entropy, target_col_name = 'class'):
    
    assert type(att_name)==str, "Expected type str for att_name!"
    
    #return non-duplicate values from the column and it's frequency
    vals,counts= np.unique(data[att_name],return_counts=True)
    w_entropy = 0
    for i in range(len(vals)):
        #filter out the class with its corresponding attributes w.r.t features
        att = data.where(data[att_name]==vals[i]).dropna()[target_col_name]
        w_entropy += (counts[i]/np.sum(counts))*entropy(att, n_class)
    return total_entropy- w_entropy

# Data Processing

In [3]:
input_path = 'car.csv'
input_ds = pd.read_csv(input_path, header = None)
n_cols = len(input_ds.columns)
print("Number of columns:", n_cols)

Number of columns: 7


## Naming columns 

In [4]:
#initiate empty list for column name
col_name = []

for i in range(n_cols):
    #last column is the target value
    if (i == n_cols-1):
        col_name.append('class')
    else:
        col_name.append('att{}'.format(i))
        
print(col_name)

['att0', 'att1', 'att2', 'att3', 'att4', 'att5', 'class']


In [5]:
#assigning column names to data set
input_ds.columns = col_name
print (input_ds.head())

    att0   att1 att2 att3   att4  att5  class
0  vhigh  vhigh    2    2  small   low  unacc
1  vhigh  vhigh    2    2  small   med  unacc
2  vhigh  vhigh    2    2  small  high  unacc
3  vhigh  vhigh    2    2    med   low  unacc
4  vhigh  vhigh    2    2    med   med  unacc


In [6]:
for headings in input_ds:
    print("Attributes in {} is: {}".format(headings, set(input_ds[headings])))
    
n_class = len(set(input_ds['class']))
print("Number of classes: ", n_class)

Attributes in att0 is: {'vhigh', 'med', 'low', 'high'}
Attributes in att1 is: {'vhigh', 'med', 'low', 'high'}
Attributes in att2 is: {'2', '5more', '4', '3'}
Attributes in att3 is: {'2', '4', 'more'}
Attributes in att4 is: {'med', 'small', 'big'}
Attributes in att5 is: {'low', 'med', 'high'}
Attributes in class is: {'vgood', 'acc', 'good', 'unacc'}
Number of classes:  4


-----------------------------------------

# Simulation for few steps

## First step

In [7]:
#sanity check for entropy for first step 
total_entropy = entropy(input_ds['class'], n_class)
print(total_entropy)

0.6028704850060875


In [8]:
features = col_name[:n_cols-1] #ommit the class feature
print(features)

['att0', 'att1', 'att2', 'att3', 'att4', 'att5']


In [9]:
#loop through the feature columns to calculate gain
info_gain = [gain(input_ds, feature, n_class) for feature in features]
highest_gain_index = np.argmax(info_gain)
best_feature = features[highest_gain_index]

print(best_feature)

att5


In [10]:
#remove the best column of the best feature
features = [i for i in features if i!=best_feature]
print(features)

['att0', 'att1', 'att2', 'att3', 'att4']


In [11]:
#split the data by matching the value with respect to non-duplicate values
# in the feature column
sub_data_list = []
best_feature_col = input_ds[best_feature]
for value in np.unique(best_feature_col):
    sub_data = input_ds.where(best_feature_col==value).dropna()
#     print(sub_data)  
    if len(np.unique(sub_data['class']))<=1:
        ent = entropy(sub_data['class'], n_class)
        target = np.unique(sub_data['class'])[0]
        print("entropy = {}, feature = {}, value = {}, class = {}".format(ent, best_feature, value, target))
        #become tree
    else:
        ent = entropy(sub_data['class'], n_class)
        print("entropy = {}, feature = {}, value = {}".format(ent, best_feature, value))
        sub_data_list.append(sub_data)
        #grow tree

entropy = 0.8077559247898061, feature = att5, value = high
entropy = 0.0, feature = att5, value = low, class = unacc
entropy = 0.607578995397061, feature = att5, value = med


In [12]:
sub_data_list[1]

,att0,att1,att2,att3,att4,att5,class
1,vhigh,vhigh,2,2,small,med,unacc
4,vhigh,vhigh,2,2,med,med,unacc
7,vhigh,vhigh,2,2,big,med,unacc
10,vhigh,vhigh,2,4,small,med,unacc
13,vhigh,vhigh,2,4,med,med,unacc
16,vhigh,vhigh,2,4,big,med,unacc
19,vhigh,vhigh,2,more,small,med,unacc
22,vhigh,vhigh,2,more,med,med,unacc
25,vhigh,vhigh,2,more,big,med,unacc
28,vhigh,vhigh,3,2,small,med,unacc


In [13]:
#loop through the feature columns to calculate gain
info_gain = [gain(sub_data_list[1], feature, n_class) for feature in features]
highest_gain_index = np.argmax(info_gain)
best_feature = features[highest_gain_index]

print(best_feature)

att3


In [14]:
#split the data by matching the value with respect to non-duplicate values
# in the feature column
sub_data_list = []
best_feature_col = input_ds[best_feature]
for value in np.unique(best_feature_col):
    sub_data = input_ds.where(best_feature_col==value).dropna()
#     print(sub_data)  
    if len(np.unique(sub_data['class']))<=1:
        ent = entropy(sub_data['class'], n_class)
        target = np.unique(sub_data['class'])[0]
        print("entropy = {}, feature = {}, value = {}, class = {}".format(ent, best_feature, value, target))
        #become tree
    else:
        ent = entropy(sub_data['class'], n_class)
        print("entropy = {}, feature = {}, value = {}".format(ent, best_feature, value))
        sub_data_list.append(sub_data)
        #grow tree

entropy = 0.0, feature = att3, value = 2, class = unacc
entropy = 0.7403599651474037, feature = att3, value = 4
entropy = 0.738757044860997, feature = att3, value = more


---------------------------

In [36]:
def decision_tree_id3(original_data, data, features, n_class, target_col_name = 'class', parent_node_class = None):
    '''
    This function is a recursive function for calculating decision tree with ID3 algorithm.
    
    @param original_data: Orignial dataset that includes all the feature columns
    @param data: Data that the algorithm is currently running
    @param features: A list containing feature column names
    @param n_class: Number of class as log base
    @param target_col_name: Column name where the target values are stored.
    @param parent_node_class: The best target feature value will be stored
    
    return: tree
    '''
    #specify stopping criteria
    
    #if the values in the class column is the same, return the value
    if len(np.unique(data[target_col_name]))<=1:
        return np.unique(data[target_col_name])[0]
    #if dataset is empty, output the value in the target column with the 
    # highest occurence 
    elif len(data)==0:
        #axis 1 is the list where the counts are stored
        highest_count_index = np.argmax(np.unique(original_data[target_col_name],return_counts=True)[1])
        return np.unique(original_data[target_col_name])[highest_count_index]
    #if feature space is empty, output parent_node_class
    elif len(features)==0:
        return parent_node_class
    #compute decision tree
    else: 
        #set the mode of the target feature value of the current node as the default node
        highest_count_index = np.argmax(np.unique(data[target_col_name],return_counts=True)[1])
        parent_node_class = np.unique(data[target_col_name])[highest_count_index]
        
        #loop across the feature columns for highest gain
        info_gain = [gain(data, feature, n_class) for feature in features]
        #retrieve the index of the highest gain so that we can know the feature
        highest_gain_index = np.argmax(info_gain)
        best_feature = features[highest_gain_index]
        
        #Assign name to the root or parent nodes
        tree = {best_feature:{}}
        
        #remove the best feature from the previous step
        features = [i for i in features if i!= best_feature]
        
        for value in np.unique(data[best_feature]):
            sub_data = data.where(data[best_feature]==value).dropna()
            
            #recursive starts here
            sub_tree = decision_tree_id3(data, sub_data, features, n_class, parent_node_class)
            #add sub tree under the root node
            tree[best_feature][value] = subtree
            
        return tree

In [81]:
data= input_ds
target_col_name ='class'
if len(np.unique(data[target_col_name]))<=1:
    np.unique(data[target_col_name])[0]

In [37]:
features = col_name[:n_cols-1]
test = decision_tree_id3(input_ds, input_ds, features, n_class)

KeyError: 'unacc'

In [65]:
for feature in features:
    info_gain = gain(input_ds, feature, n_class)
    print(info_gain)
highest_gain_index = np.argmax(info_gain)
max_gain_feature = features[highest_gain_index]

0.03685197346074287
0.002242858313315943
0.015004070623802601


In [66]:
tree = {max_gain_feature:{}}
print(tree)

{'att1': {}}


In [56]:
np.unique(input_ds['class'],return_counts=True)

(array(['acc', 'good', 'unacc', 'vgood'], dtype=object),
 array([ 384,   69, 1210,   65]))

---------------

In [24]:
def entropy(target_col):
    """
    Calculate the entropy of a dataset.
    The only parameter of this function is the target_col parameter which specifies the target column
    """
    elements,counts = np.unique(target_col,return_counts = True)
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy
################### 
    
###################
def InfoGain(data,split_attribute_name,target_name="class"):
    """
    Calculate the information gain of a dataset. This function takes three parameters:
    1. data = The dataset for whose feature the IG should be calculated
    2. split_attribute_name = the name of the feature for which the information gain should be calculated
    3. target_name = the name of the target feature. The default for this example is "class"
    """    
    #Calculate the entropy of the total dataset
    total_entropy = entropy(data[target_name])
    
    ##Calculate the entropy of the dataset
    
    #Calculate the values and the corresponding counts for the split attribute 
    vals,counts= np.unique(data[split_attribute_name],return_counts=True)
    
    #Calculate the weighted entropy
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    
    #Calculate the information gain
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain
       
###################
###################
def ID3(data,originaldata,features,target_attribute_name="class",parent_node_class = None):
    """
    ID3 Algorithm: This function takes five paramters:
    1. data = the data for which the ID3 algorithm should be run --> In the first run this equals the total dataset
 
    2. originaldata = This is the original dataset needed to calculate the mode target feature value of the original dataset
    in the case the dataset delivered by the first parameter is empty
    3. features = the feature space of the dataset . This is needed for the recursive call since during the tree growing process
    we have to remove features from our dataset --> Splitting at each node
    4. target_attribute_name = the name of the target attribute
    5. parent_node_class = This is the value or class of the mode target feature value of the parent node for a specific node. This is 
    also needed for the recursive call since if the splitting leads to a situation that there are no more features left in the feature
    space, we want to return the mode target feature value of the direct parent node.
    """   
    #Define the stopping criteria --> If one of this is satisfied, we want to return a leaf node#
    
    #If all target_values have the same value, return this value
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
    
    #If the dataset is empty, return the mode target feature value in the original dataset
    elif len(data)==0:
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name],return_counts=True)[1])]
    
    #If the feature space is empty, return the mode target feature value of the direct parent node --> Note that
    #the direct parent node is that node which has called the current run of the ID3 algorithm and hence
    #the mode target feature value is stored in the parent_node_class variable.
    
    elif len(features) ==0:
        return parent_node_class
    
    #If none of the above holds true, grow the tree!
    
    else:
        #Set the default value for this node --> The mode target feature value of the current node
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])]
        
        #Select the feature which best splits the dataset
        item_values = [InfoGain(data,feature,target_attribute_name) for feature in features] #Return the information gain values for the features in the dataset
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        
        #Create the tree structure. The root gets the name of the feature (best_feature) with the maximum information
        #gain in the first run
        tree = {best_feature:{}}
        
        
        #Remove the feature with the best inforamtion gain from the feature space
        features = [i for i in features if i != best_feature]
        
        #Grow a branch under the root node for each possible value of the root node feature
        
        for value in np.unique(data[best_feature]):
            value = value
            #Split the dataset along the value of the feature with the largest information gain and therwith create sub_datasets
            sub_data = data.where(data[best_feature] == value).dropna()
            
            #Call the ID3 algorithm for each of those sub_datasets with the new parameters --> Here the recursion comes in!
            subtree = ID3(sub_data,data,features,target_attribute_name,parent_node_class)
            
            #Add the sub tree, grown from the sub_dataset to the tree under the root node
            tree[best_feature][value] = subtree
            
        return(tree) 

In [27]:
tree = ID3(input_ds, input_ds, input_ds.columns[:-1])

In [30]:
print(tree)

{'att5': {'high': {'att3': {'2': 'unacc', '4': {'att0': {'high': {'att1': {'high': 'acc', 'low': 'acc', 'med': 'acc', 'vhigh': 'unacc'}}, 'low': {'att1': {'high': {'att4': {'big': 'vgood', 'med': {'att2': {'2': 'acc', '3': 'acc', '4': 'vgood', '5more': 'vgood'}}, 'small': 'acc'}}, 'low': {'att4': {'big': 'vgood', 'med': {'att2': {'2': 'good', '3': 'good', '4': 'vgood', '5more': 'vgood'}}, 'small': 'good'}}, 'med': {'att4': {'big': 'vgood', 'med': {'att2': {'2': 'good', '3': 'good', '4': 'vgood', '5more': 'vgood'}}, 'small': 'good'}}, 'vhigh': 'acc'}}, 'med': {'att1': {'high': 'acc', 'low': {'att4': {'big': 'vgood', 'med': {'att2': {'2': 'good', '3': 'good', '4': 'vgood', '5more': 'vgood'}}, 'small': 'good'}}, 'med': {'att4': {'big': 'vgood', 'med': {'att2': {'2': 'acc', '3': 'acc', '4': 'vgood', '5more': 'vgood'}}, 'small': 'acc'}}, 'vhigh': 'acc'}}, 'vhigh': {'att1': {'high': 'unacc', 'low': 'acc', 'med': 'acc', 'vhigh': 'unacc'}}}}, 'more': {'att0': {'high': {'att1': {'high': {'att2'